In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time

# Action1：汽车投诉信息采集：

In [2]:
# 获取页面信息
def get_page_content(url):
    # 获取页面的内容
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    html=requests.get(url,headers=headers,timeout=10)
    content = html.text
    # 通过content创建BeautifulSoup对象
    soup = bs(content, 'html.parser', from_encoding='utf-8')
    return soup

In [3]:
#拾取页面内投诉信息
def get_data(soup):
    # 找到完整的投诉信息
    temp = soup.find('div',class_="tslb_b")
    # 创建DataFrame
    df = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status'])
    tr_list = temp.find_all('tr')
    #按tr读取行信息
    for tr in tr_list:
        td_list=tr.find_all('td')
        if len(td_list)>0:
            #按td读取单元格信息，并组成pd
            id,brand,car_model,type,desc,problem,datetime,status=\
            td_list[0].text,td_list[1].text,td_list[2].text,td_list[3].text,\
            td_list[4].text,td_list[5].text,td_list[6].text,td_list[7].text,
            temp={}
            temp['id']=id
            temp['brand']=brand
            temp['car_model']=car_model
            temp['type']=type
            temp['desc']=desc
            temp['problem']=problem
            temp['datetime']=datetime
            temp['status']=status
            df=df.append(temp,ignore_index=True)
    return df

In [4]:
#获取前page_num页数据
page_num=20
base_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-'
result = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status'])
for i in range(page_num):
    #自动生成网址
    url=base_url+str(i+1)+'.shtml'
    soup=get_page_content(url)
    df=get_data(soup)
    result=result.append(df)
    #等待
    time.sleep(5)
print(result)
result.to_csv('car_complain.csv',index=False)
result.to_excel('car_complain.xlsx',index=False)

d:\anaconda3\lib\site-packages\bs4\__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


        id    brand car_model                          type  \
0   571251    一汽-大众       高尔夫            2016款 1.6L  自动 舒适型   
1   571247     华晨宝马   宝马5系新能源         2020款 530Le 里程升级版 先锋版   
2   571246     众泰汽车      大迈X7             2018款 2.0T 自动 尊贵型   
3   571243     奇瑞汽车       瑞虎8          2019款 1.6TGDI 自动 精英型   
4   571242  一汽-大众奥迪      奥迪A3  2016款 Sportback 1.4T 双离合 风尚型   
..     ...      ...       ...                           ...   
25  569923     东风本田   INSPIRE        2019款 锐·混动 2.0L 净尚版 国V   
26  569920     吉利汽车      远景X6            2020款 1.4T CVT 尊贵型   
27  569917     华晨宝马      宝马3系             2021款 325Li M运动套装   
28  569916     东风日产        轩逸       2012款 1.6L XE 自动 舒适版 经典   
29  569915     长安福特       蒙迪欧      2013款 1.5T 自动 GTDi180时尚型   

                    desc                       problem    datetime status  
0    一汽大众高尔夫变速箱报故障码维修未解决                          B96,  2021-02-08   信息审核  
1   华晨宝马5系新能源天窗异响反复维修未解决                     H89,J298,  2021-02-08   信息审核  
2      众泰大迈X7永久